<a href="https://colab.research.google.com/github/sowmi1407/Task_3_implementation/blob/main/LayoutLM_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade datasets fsspec aiohttp


  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [2]:
from datasets import load_dataset

ds = load_dataset("dvgodoy/rvl_cdip_mini", cache_dir="./new_hf_cache")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/341M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/42.4M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/41.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3200 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [9]:
def print_label_range(dataset):
    labels = [example['label'] for example in dataset]
    unique_labels = set(labels)
    print("Unique labels:", sorted(unique_labels))
    print("Min label:", min(unique_labels))
    print("Max label:", max(unique_labels))


print("Train:")
print_label_range(ds['train'])
print("Validation:")
print_label_range(ds['validation'])
print("Test:")
print_label_range(ds['test'])

Train:
Unique labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Min label: 0
Max label: 15
Validation:
Unique labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Min label: 0
Max label: 15
Test:
Unique labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Min label: 0
Max label: 15


In [10]:
def keep_first_three_classes(example):
    return example["label"] in [0, 1, 2]


train_filtered = ds["train"].filter(keep_first_three_classes)
val_filtered = ds["validation"].filter(keep_first_three_classes)
test_filtered = ds["test"].filter(keep_first_three_classes)

Filter:   0%|          | 0/3200 [00:00<?, ? examples/s]

Filter:   0%|          | 0/400 [00:00<?, ? examples/s]

Filter:   0%|          | 0/400 [00:00<?, ? examples/s]

In [11]:
print(len(test_filtered))

65


In [4]:
import torch
from transformers import LayoutLMTokenizerFast

tokenizer = LayoutLMTokenizerFast.from_pretrained("microsoft/layoutlm-base-uncased")

def encode_example(example, max_seq_length=512, pad_token_box=[0, 0, 0, 0]):
    words = example['ocr_words']
    boxes = example['word_boxes']
    label = example['label']

    encoding = tokenizer(
        words,
        is_split_into_words=True,
        padding='max_length',
        truncation=True,
        max_length=max_seq_length,
        return_tensors='pt'
    )


    word_ids = encoding.word_ids(batch_index=0)
    bbox = []
    for word_idx in word_ids:
        if word_idx is None:
            bbox.append(pad_token_box)
        else:
            bbox.append(boxes[word_idx])

    return {
        'input_ids': encoding['input_ids'].squeeze(0),
        'attention_mask': encoding['attention_mask'].squeeze(0),
        'token_type_ids': encoding['token_type_ids'].squeeze(0),
        'bbox': torch.tensor(bbox, dtype=torch.long),
        'label': label
    }


tokenizer_config.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

In [5]:
encoded_train = train_filtered.map(encode_example)
encoded_val = val_filtered.map(encode_example)
encoded_test = test_filtered.map(encode_example)


Map:   0%|          | 0/585 [00:00<?, ? examples/s]

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

In [6]:
columns = ['input_ids', 'attention_mask', 'token_type_ids', 'bbox', 'label']
encoded_train.set_format(type='torch', columns=columns)
encoded_val.set_format(type='torch', columns=columns)
encoded_test.set_format(type='torch', columns=columns)

In [7]:
train_dataset = encoded_train
val_dataset = encoded_val
test_dataset = encoded_test

In [8]:
def check_labels_greater_than_3(dataset):
    labels = [example['label'] for example in dataset]
    greater_than_3 = [label for label in labels if label > 2]
    if greater_than_3:
        print("Labels greater than 3 found:", set(greater_than_3))
    else:
        print("All labels are 3 or less.")

# Example usage:
check_labels_greater_than_3(train_dataset)
check_labels_greater_than_3(val_dataset)
check_labels_greater_than_3(test_dataset)

All labels are 3 or less.
All labels are 3 or less.
All labels are 3 or less.


In [11]:
from transformers import LayoutLMForSequenceClassification

model = LayoutLMForSequenceClassification.from_pretrained(
    "microsoft/layoutlm-base-uncased",
    num_labels=3
)

Some weights of LayoutLMForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW


def train_model(model, train_dataset, val_dataset, test_dataset, device, num_train_epochs=10, batch_size=4, learning_rate=4e-5):

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    optimizer = AdamW(model.parameters(), lr=learning_rate)
    model.to(device)

    for epoch in range(num_train_epochs):
        print(f"\nEpoch: {epoch + 1}/{num_train_epochs}")
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for batch in train_dataloader:
            input_ids = batch["input_ids"].to(device)
            bbox = batch["bbox"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(
                input_ids=input_ids,
                bbox=bbox,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels
            )
            loss = outputs.loss
            logits = outputs.logits

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * input_ids.size(0)
            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / total
        train_acc = correct / total
        print(f"Training loss: {train_loss:.4f}, accuracy: {train_acc:.4f}")

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for batch in val_dataloader:
                input_ids = batch["input_ids"].to(device)
                bbox = batch["bbox"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                token_type_ids = batch["token_type_ids"].to(device)
                labels = batch["label"].to(device)

                outputs = model(
                    input_ids=input_ids,
                    bbox=bbox,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    labels=labels
                )
                loss = outputs.loss
                logits = outputs.logits

                val_loss += loss.item() * input_ids.size(0)
                preds = torch.argmax(logits, dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= val_total
        val_acc = val_correct / val_total
        print(f"Validation loss: {val_loss:.4f}, accuracy: {val_acc:.4f}")

    # Testing
    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch["input_ids"].to(device)
            bbox = batch["bbox"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            labels = batch["label"].to(device)

            outputs = model(
                input_ids=input_ids,
                bbox=bbox,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels
            )
            logits = outputs.logits

            preds = torch.argmax(logits, dim=1)
            test_correct += (preds == labels).sum().item()
            test_total += labels.size(0)

    test_acc = test_correct / test_total
    print(f"\nTest accuracy: {test_acc:.4f}")

    return model

# Example usage:
model = LayoutLMForSequenceClassification.from_pretrained('microsoft/layoutlm-base-uncased', num_labels=16)
trained_model = train_model(model, train_dataset, val_dataset, test_dataset, device='cuda' if torch.cuda.is_available() else 'cpu')


Some weights of LayoutLMForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch: 1/10
